# Multilayer Perceptron Classifier

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import statistics
import random as r

## Split the Data into Train and Test

In [2]:
data = pd.read_csv('data/segmented_data/data_2000_2020.csv')
data = data.drop(['Unnamed: 0'], axis=1)

In [3]:
data.head()

fips    state   county  wins  poverty_rate    population  \
0  1005  Alabama  Barbour     3     36.383333  24581.000000   
1  1023  Alabama  Choctaw     2     35.100000  16119.333333   
2  1033  Alabama  Colbert     2     19.516667  51655.000000   
3  1035  Alabama  Conecuh     3     37.466667  15075.500000   
4  1071  Alabama  Jackson     2     23.116667  46857.333333   

   persons_in_poverty  rural_urban_code  urban_influence_code  metro  ...  \
0         8902.166667               6.0                   6.0    0.0  ...   
1         5815.500000               9.0                  10.0    0.0  ...   
2         9884.666667               3.0                   2.0    1.0  ...   
3         5833.833333               7.0                  11.0    0.0  ...   
4        10124.333333               6.0                   5.0    0.0  ...   

   percent_black_male  percent_black_female  percent_native_american_male  \
0           25.180116             22.500706                      0.377976   
1           19.637383             22.927498                      0.103233   
2            7.431415              8.769940                      0.310621   
3           21.227098             25.178039                      0.243664   
4            1.689644              1.766228                      0.782550   

   percent_native_american_female  percent_asian_male  percent_asian_female  \
0                        0.268640            0.218840              0.220066   
1                        0.064972            0.045841              0.096745   
2                        0.299706            0.233118              0.324036   
3                        0.200267            0.056918              0.123358   
4                        0.812930            0.218962              0.283917   

   percent_hispanic_male  percent_hispanic_female  cumulative_confirmed_cases  \
0               2.688941                 2.099242                      298307   
1               0.362755                 0.408992                      108088   
2               1.396695                 1.226652                      746845   
3               1.003296                 0.753020                      166637   
4               1.498113                 1.293818                      776358   

   cumulative_deaths  
0               4787  
1               4438  
2              11653  
3               3646  
4               7270  

[5 rows x 34 columns]

In [4]:
# Create a new column that describes bellwether status

data['bellwether'] = ''
for i, v in enumerate(data.wins):
    if v > 4:
        data.iloc[i, -1] = 1
    elif v <= 4:
        data.iloc[i, -1] = 0

In [5]:
# Train/test split

X = data.drop(['fips', 'state', 'county', 'wins', 'bellwether'], axis=1)
y = data.bellwether

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25,random_state=0)

In [6]:
# Standardize the inputs

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Convert integers to floats

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [8]:
print('X_train: {0}\nX_test: {1}\ny_train: {2}\ny_test: {3}'.format(X_train.shape, X_test.shape, len(y_train), len(y_test)))

X_train: (531, 30)
X_test: (177, 30)
y_train: 531
y_test: 177


In [9]:
# Balance train and test Sets using oversampling (2:1 ratio)

from collections import Counter
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy=0.5)

X_over_train, y_over_train = oversample.fit_resample(X_train, y_train)
print('y_over_train: {0}'.format(Counter(y_over_train)))

X_over_test, y_over_test = oversample.fit_resample(X_test, y_test)
print('y_over_test: {0}'.format(Counter(y_over_test)))

y_over_train: Counter({0.0: 432, 1.0: 216})
y_over_test: Counter({0.0: 136, 1.0: 68})


## Multilayer Perceptron with ReLU Activation

In [72]:
from tensorflow import keras
from tensorflow.keras import layers
from keras import optimizers
from keras.models import Model, Sequential
from keras.layers import Dense, Activation
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, BatchNormalization, Conv1D, Conv2D, Flatten, MaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import classification_report

In [11]:
X_over_train.shape[1]*2

60

In [12]:
# define a fully connected feedforward keras model
model = Sequential()
model.add(Dense(X_over_train.shape[1], input_dim=X_over_train.shape[1], activation='relu'))
model.add(Dense(X_over_train.shape[1]*2, activation='relu'))
model.add(Dense(X_over_train.shape[1]*2, activation='relu'))
model.add(Dense(X_over_train.shape[1]*2, activation='relu'))
model.add(Dense(X_over_train.shape[1]*2, activation='relu'))
model.add(Dense(X_over_train.shape[1]*2, activation='relu'))
model.add(Dense(X_over_train.shape[1]*2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X_over_train, y_over_train, epochs=50, batch_size=10, verbose=1)

Epoch 1/50
65/65 [==============================] - 1s 751us/step - loss: 0.6244 - accuracy: 0.6641
Epoch 2/50
65/65 [==============================] - 0s 766us/step - loss: 0.5131 - accuracy: 0.7302
Epoch 3/50
65/65 [==============================] - 0s 735us/step - loss: 0.5363 - accuracy: 0.7042
Epoch 4/50
65/65 [==============================] - 0s 751us/step - loss: 0.4889 - accuracy: 0.7442
Epoch 5/50
65/65 [==============================] - 0s 719us/step - loss: 0.4634 - accuracy: 0.7761
Epoch 6/50
65/65 [==============================] - 0s 798us/step - loss: 0.3937 - accuracy: 0.8215
Epoch 7/50
65/65 [==============================] - 0s 829us/step - loss: 0.3589 - accuracy: 0.8408
Epoch 8/50
65/65 [==============================] - 0s 798us/step - loss: 0.3622 - accuracy: 0.8532
Epoch 9/50
65/65 [==============================] - 0s 766us/step - loss: 0.3302 - accuracy: 0.8632
Epoch 10/50
65/65 [==============================] - 0s 728us/step - loss: 0.2632 - accuracy: 0.8954

In [13]:
# evaluate the keras model
output, accuracy = model.evaluate(X_over_test, y_over_test)

7/7 [==============================] - 0s 667us/step - loss: 1.8686 - accuracy: 0.7010


In [14]:
# visualize preceptron

import pydotplus

keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [15]:
# Print accuracy, error, matrix, and classification report

y_pred_class = model.predict(X_over_train)
y_pred_class = np.where(y_pred_class > .5, 1, 0)
training_accuracy = metrics.accuracy_score(y_over_train, y_pred_class)
training_error = 1 - training_accuracy

y_pred_class = model.predict(X_over_test)
y_pred_class = np.where(y_pred_class > .5, 1, 0)
testing_accuracy = metrics.accuracy_score(y_over_test, y_pred_class)
testing_error = 1 - testing_accuracy

print('MLP accuracy: {0}\nTest error: {1}\n'.format(testing_accuracy, testing_error))
print('Confusion Matrix:\n[[TN  FP]\n[ FN  TP]]\n{0}\n'.format(metrics.confusion_matrix(y_over_test, y_pred_class)))

report = classification_report(y_over_test.values, y_pred_class, output_dict=True)
print('Classification Report:\n' + classification_report(y_over_test.values, y_pred_class))

MLP accuracy: 0.7009803921568627
Test error: 0.2990196078431373

Confusion Matrix:
[[TN  FP]
[ FN  TP]]
[[117  19]
 [ 42  26]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.74      0.86      0.79       136
         1.0       0.58      0.38      0.46        68

    accuracy                           0.70       204
   macro avg       0.66      0.62      0.63       204
weighted avg       0.68      0.70      0.68       204



In [16]:
model.predict(X_over_test)

array([[2.54470110e-03],
       [9.91343021e-01],
       [8.21241736e-03],
       [8.31388718e-08],
       [2.24488552e-08],
       [3.22359800e-03],
       [3.04877758e-04],
       [7.97510147e-04],
       [1.00000000e+00],
       [4.15354967e-04],
       [4.01726140e-07],
       [6.53237104e-04],
       [3.41527998e-01],
       [1.15476778e-05],
       [9.33876349e-13],
       [1.39385462e-04],
       [3.58875081e-06],
       [5.99238276e-03],
       [6.10453662e-06],
       [9.44576740e-01],
       [7.36355782e-04],
       [6.31035505e-07],
       [6.59745097e-01],
       [2.09857821e-02],
       [5.24255633e-03],
       [2.04634666e-03],
       [4.09509653e-07],
       [5.00455499e-03],
       [1.77281797e-02],
       [2.18321657e-05],
       [8.72894981e-15],
       [1.50799751e-04],
       [3.74365669e-07],
       [1.29191935e-10],
       [7.14031458e-01],
       [7.76392631e-11],
       [1.71120763e-02],
       [9.44381573e-10],
       [1.39364600e-03],
       [8.31088424e-03],


In [17]:
y_pred_class = model.predict(X_over_test)
y_pred_class = np.where(y_pred_class > .5, 1, 0)
y_pred_class

array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
    

## Hyperparameter Optimization

In [18]:
# select parameter ranges

hidden_layers = [1, 6, 12]
dimensions_per_layer = [X_over_train.shape[1]*2]
hidden_layer_activation = ['relu', 'selu']
output_layer_activation = ['sigmoid']
loss_function = ['binary_crossentropy', 'mean_squared_error']
optimizer = ['adam', 'adamax', 'nadam']
epochs = [100, 175, 250]
batch_size = [10, 50, 100]

In [19]:
# define variables to be measured

overall_test_accuracy = []
overall_precision = []
overall_recall = []
overall_f1_score = []

In [20]:
# define the model to be run

def run_model(layers, dim, hl_act, ol_act, lf, opt, e, b):
    
    # define the seed
    seed = 0
    
    # create a new function that appends bellwether status to a df
    def create_label(data):
        data['bellwether'] = ''
        for i, v in enumerate(data.wins):
            if v > 4:
                data.iloc[i, -1] = 1
            elif v <= 4:
                data.iloc[i, -1] = 0
    
    # create a new function that samples from a df
    def sample(df, n, split):
        sample_list_60 = list(df['Unnamed: 0'].sample(n=424, random_state=seed))
        sample_list_80 = list(df['Unnamed: 0'].sample(n=566, random_state=seed))
        if split == 'train':
            for i, v in enumerate(df['Unnamed: 0']):
                if v not in sample_list_60:
                    df = df.drop(i)
        if split == 'val':
            for i, v in enumerate(df['Unnamed: 0']):
                if v in sample_list_60 and v in sample_list_80:
                    df = df.drop(i)
                if v not in sample_list_80:
                    df = df.drop(i)
        if split == 'test':
            for i, v in enumerate(df['Unnamed: 0']):
                if v in sample_list_80:
                    df = df.drop(i)
        return df
    
    # define inner variables to be appended
    test_accuracy = []
    precision = []
    recall = []
    f1_score = []

    # get the average of 5 runs
    for i in range(5):

        seed = r.randint(1, 1000)
        
        # create train set (60%), validation set (20%), and test set (20%)
        X_train = pd.read_csv('data/segmented_data/data_2000_2020.csv')
        X_train = sample(X_train, 424, 'train')
        X_train = X_train.drop(['Unnamed: 0', 'fips', 'state', 'county', 'wins'], axis=1)
        y_train = pd.read_csv('data/segmented_data/data_2000_2020.csv')
        y_train = sample(y_train, 424, 'train')
        create_label(y_train)
        y_train = y_train.bellwether

        val_set = pd.read_csv('data/segmented_data/data_2000_2020.csv')
        val_set = sample(val_set, 142, 'val')
        create_label(val_set)
        X_val = np.array(val_set.drop(['Unnamed: 0', 'fips', 'state', 'county', 'wins', 'bellwether'], axis=1))
        X_val = X_val.astype('float32')
        y_val = np.array(val_set.bellwether).astype('float32')
        y_val = y_val.astype('float32')
        val_set = (X_val, y_val)

        X_test = pd.read_csv('data/segmented_data/data_2000_2020.csv')
        X_test = sample(X_test, 142, 'test')
        X_test = X_test.drop(['Unnamed: 0', 'fips', 'state', 'county', 'wins'], axis=1)
        y_test = pd.read_csv('data/segmented_data/data_2000_2020.csv')
        y_test = sample(y_test, 142, 'test')
        create_label(y_test)
        y_test = y_test.bellwether
        
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        y_train = y_train.astype('float32')
        y_test = y_test.astype('float32')
        
        oversample = RandomOverSampler(sampling_strategy=0.5)
        X_over_train, y_over_train = oversample.fit_resample(X_train, y_train)
        X_over_test, y_over_test = oversample.fit_resample(X_test, y_test)
        
        # define a fully connected feedforward keras model
        model = Sequential()
        model.add(Dense(X_over_train.shape[1], input_dim=X_over_train.shape[1], activation=hl_act))
        for x in range(layers):
            model.add(Dense(dim, activation=hl_act))
        model.add(Dense(1, activation=ol_act))

        # compile the keras model
        model.compile(loss=lf, optimizer=opt, metrics=['accuracy'])

        # fit the keras model on the dataset
        model.fit(X_over_train, y_over_train, epochs=e, batch_size=b, verbose=0, validation_split=0.1, callbacks=[])

        # evaluate the keras model
        loss, accuracy = model.evaluate(X_over_test, y_over_test, verbose=0)

        y_pred_class = model.predict(X_over_train)
        y_pred_class = np.where(y_pred_class > .5, 1, 0)
        training_accuracy = metrics.accuracy_score(y_over_train, y_pred_class)
        training_error = 1 - training_accuracy

        y_pred_class = model.predict(X_over_test)
        y_pred_class = np.where(y_pred_class > .5, 1, 0)
        testing_accuracy = metrics.accuracy_score(y_over_test, y_pred_class)
        testing_error = 1 - testing_accuracy

        report = classification_report(y_over_test.values, y_pred_class, output_dict=True)
        matrix = metrics.confusion_matrix(y_over_test, y_pred_class)

        # append the data
        test_accuracy.append(accuracy)
        precision.append(report['1.0']['precision'])
        recall.append(report['1.0']['recall'])
        f1_score.append(report['1.0']['f1-score'])

    overall_test_accuracy.append(statistics.mean(test_accuracy))
    overall_precision.append(statistics.mean(precision))
    overall_recall.append(statistics.mean(recall))
    overall_f1_score.append(statistics.mean(f1_score))

In [ ]:
# run the model on various parameters (1/3)

for dim in dimensions_per_layer:
    for hl_act in hidden_layer_activation:
        for ol_act in output_layer_activation:
            for lf in loss_function:
                for opt in optimizer:
                    for e in epochs:
                        for b in batch_size:
                            run_model(1, dim, hl_act, ol_act, lf, opt, e, b)

In [66]:
len(overall_test_accuracy)

432

In [67]:
# run the model on various parameters (2/3)

for dim in dimensions_per_layer:
    for hl_act in hidden_layer_activation:
        for ol_act in output_layer_activation:
            for lf in loss_function:
                for opt in optimizer:
                    for e in epochs:
                        for b in batch_size:
                            run_model(6, dim, hl_act, ol_act, lf, opt, e, b)

In [68]:
len(overall_test_accuracy)

540

In [69]:
# run the model on various parameters (3/3)

for dim in dimensions_per_layer:
    for hl_act in hidden_layer_activation:
        for ol_act in output_layer_activation:
            for lf in loss_function:
                for opt in optimizer:
                    for e in epochs:
                        for b in batch_size:
                            run_model(12, dim, hl_act, ol_act, lf, opt, e, b)

c:\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [82]:
len(overall_test_accuracy)

324

In [89]:
# append the parameter combinations to a list

list_hidden_layers = []
list_dimensions_per_layer = []
list_hidden_layer_activation = []
list_output_layer_activation = []
list_loss_function = []
list_optimizer = []
list_epochs = []
list_batch_size = []

for layers in hidden_layers:
    for dim in dimensions_per_layer:
        for hl_act in hidden_layer_activation:
            for ol_act in output_layer_activation:
                for lf in loss_function:
                    for opt in optimizer:
                        for e in epochs:
                            for b in batch_size:
                                list_hidden_layers.append(layers)
                                list_dimensions_per_layer.append(dim)
                                list_hidden_layer_activation.append(hl_act)
                                list_output_layer_activation.append(ol_act)
                                list_loss_function.append(lf)
                                list_optimizer.append(opt)
                                list_epochs.append(e)
                                list_batch_size.append(b)
                                
# create a dataframe to hold the data
column_dict = {
    'hidden_layers': list_hidden_layers, 
    'dimensions_per_layer': list_dimensions_per_layer, 
    'hidden_layer_activation': list_hidden_layer_activation, 
    'output_layer_activation': list_output_layer_activation,
    'loss_function': list_loss_function, 
    'optimizer': list_optimizer, 
    'epochs': list_epochs, 
    'batch_size': list_batch_size, 
    'test_accuracy': overall_test_accuracy, 
    'precision': overall_precision, 
    'recall': overall_recall, 
    'f1_score': overall_f1_score
              }
df = pd.DataFrame(column_dict)

In [90]:
statistics.mean(overall_test_accuracy), statistics.stdev(overall_test_accuracy)

(0.687219147954458, 0.016914479618333)

In [91]:
statistics.mean(overall_precision), statistics.stdev(overall_precision)

(0.5499639815686227, 0.050325828415719025)

In [92]:
statistics.mean(overall_recall), statistics.stdev(overall_recall)

(0.31655003674037413, 0.056002216390698835)

In [93]:
statistics.mean(overall_f1_score), statistics.stdev(overall_f1_score)

(0.3939301363477067, 0.051383122554683226)

In [94]:
df.shape

(324, 12)

In [95]:
df.to_csv('data/segmented_data/mlp_optimization.csv')